Install these dependencies

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
%pip install nibabel
%pip install nilearn
%pip install SimpleITK
%pip install dcm2niix

Import dependencies

In [9]:
import gdown
import os
import shutil
import subprocess
import nibabel as nib
import nilearn.image as nimg
import numpy as np
import matplotlib.pyplot as plt
import json
from google.colab import files

Mount google drive

In [10]:
# Only run this if you use GG colab
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Download the raw dataset

In [11]:
# Google Drive file ID (from the shared link)
file_id = "1clWkwfoVhm8eSW9QVFxUWMjaRGD2XA72"
file_name = "raw_data.zip"
file_path = "/content/" + file_name

# Download the file
gdown.download(f"https://drive.google.com/uc?id={file_id}", file_name, quiet=False)
print(f"Downloaded: {file_path}")

Downloading...
From (original): https://drive.google.com/uc?id=1clWkwfoVhm8eSW9QVFxUWMjaRGD2XA72
From (redirected): https://drive.google.com/uc?id=1clWkwfoVhm8eSW9QVFxUWMjaRGD2XA72&confirm=t&uuid=97c66c36-c8df-4b34-8a75-7d87beaf14b2
To: /content/raw_data.zip

  0%|          | 0.00/4.12G [00:00<?, ?B/s]
  0%|          | 4.72M/4.12G [00:00<01:40, 40.9MB/s]
  1%|          | 32.0M/4.12G [00:00<00:24, 168MB/s] 
  1%|▏         | 57.1M/4.12G [00:00<00:20, 203MB/s]
  2%|▏         | 78.1M/4.12G [00:00<00:31, 128MB/s]
  2%|▏         | 100M/4.12G [00:00<00:26, 151MB/s] 
  3%|▎         | 118M/4.12G [00:00<00:27, 148MB/s]
  3%|▎         | 137M/4.12G [00:00<00:25, 158MB/s]
  4%|▍         | 155M/4.12G [00:01<00:35, 111MB/s]
  4%|▍         | 180M/4.12G [00:01<00:28, 140MB/s]
  5%|▍         | 206M/4.12G [00:01<00:28, 139MB/s]
  6%|▌         | 229M/4.12G [00:01<00:32, 120MB/s]
  6%|▌         | 243M/4.12G [00:01<00:38, 102MB/s]
  7%|▋         | 268M/4.12G [00:02<00:31, 121MB/s]
  7%|▋         | 283M/4.12

Downloaded: /content/raw_data.zip


Unzip these files

In [12]:
!unzip 'raw_data.zip'


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: split_dicom_data/train/data/17 - TRAN TRUNG DUNG - raw/IM000182  
  inflating: split_dicom_data/train/data/17 - TRAN TRUNG DUNG - raw/IM000183  
  inflating: split_dicom_data/train/data/17 - TRAN TRUNG DUNG - raw/IM000184  
  inflating: split_dicom_data/train/data/17 - TRAN TRUNG DUNG - raw/IM000185  
  inflating: split_dicom_data/train/data/17 - TRAN TRUNG DUNG - raw/IM000186  
  inflating: split_dicom_data/train/data/17 - TRAN TRUNG DUNG - raw/IM000187  
  inflating: split_dicom_data/train/data/17 - TRAN TRUNG DUNG - raw/IM000188  
  inflating: split_dicom_data/train/data/17 - TRAN TRUNG DUNG - raw/IM000189  
  inflating: split_dicom_data/train/data/17 - TRAN TRUNG DUNG - raw/IM000190  
  inflating: split_dicom_data/train/data/17 - TRAN TRUNG DUNG - raw/IM000191  
  inflating: split_dicom_data/train/data/17 - TRAN TRUNG DUNG - raw/IM000192  
  inflating: split_dicom_data/train/data/17 - TRAN TRUNG DUNG - raw/IM0001

Convert multiple DICOM files of each sample into a single NlfTl (input format of nnUNet)

In [13]:
# Input: folder containing all patient DICOM folders
dicom_root = r"split_dicom_data/train/data"
output_root = r"Dataset001_MyDataset/imagesTr"

# Create output folder if not exists
os.makedirs(output_root, exist_ok=True)

# Patient counter
patient_id = 1

for patient in sorted(os.listdir(dicom_root)):
    patient_path = os.path.join(dicom_root, patient)
    if os.path.isdir(patient_path):
        print(f"Converting {patient}...")

        # Temporary output (dcm2niix will dump here)
        temp_out = os.path.join(output_root, "tmp")
        os.makedirs(temp_out, exist_ok=True)

        # Run dcm2niix
        subprocess.run([
            "dcm2niix",
            "-z", "y",              # gzip compression
            "-o", temp_out,         # output folder (temporary)
            patient_path            # input DICOM folder
        ])

        # Find the converted .nii.gz file
        for f in os.listdir(temp_out):
            if f.endswith(".nii") or f.endswith(".nii.gz"):
                old_path = os.path.join(temp_out, f)

                # Rename to nnU-Net format
                new_filename = f"la_{patient_id:03d}_0000.nii.gz"
                new_path = os.path.join(output_root, new_filename)

                # Ensure gzip compressed
                if not old_path.endswith(".nii.gz"):
                    new_path_tmp = new_path.replace(".nii.gz", ".nii")
                    shutil.move(old_path, new_path_tmp)
                    subprocess.run(["gzip", new_path_tmp])
                else:
                    shutil.move(old_path, new_path)

                print(f"Renamed to {new_filename}")

        # Clean up temp
        shutil.rmtree(temp_out)

        patient_id += 1


Converting 02 - PHAM NGOC ANH - raw...
Renamed to la_001_0000.nii.gz
Converting 03 - HUYNH THI THANH XUAN - raw...
Renamed to la_002_0000.nii.gz
Converting 06 - TRAN HUNG DAO - raw...
Renamed to la_003_0000.nii.gz
Converting 07 - LUC NGOC HAI - raw...
Renamed to la_004_0000.nii.gz
Converting 09 - HOANG VAN DANG - raw...
Renamed to la_005_0000.nii.gz
Converting 10 - PHAN THI SAU - raw...
Renamed to la_006_0000.nii.gz
Converting 11 - NGUYEN THI BE - raw...
Renamed to la_007_0000.nii.gz
Converting 12 - TRAN THI TRANG - raw...
Renamed to la_008_0000.nii.gz
Converting 13 - VO HOANG VAN - raw...
Renamed to la_009_0000.nii.gz
Converting 14 - NGUYEN XUAN LOC - raw...
Renamed to la_010_0000.nii.gz
Converting 16 - NGUYEN THI NGUNG - raw...
Renamed to la_011_0000.nii.gz
Converting 17 - TRAN TRUNG DUNG - raw...
Renamed to la_012_0000.nii.gz
Converting 19 - NGUYEN QUANG TAN - raw...
Renamed to la_013_0000.nii.gz
Converting 21 - VAN DINH QUY - raw...
Renamed to la_014_0000.nii.gz
Converting 25 - HO 

Move segmentation file to new folder

In [14]:
import os
import shutil

# Input: source folder containing label .nii.gz files
seg_root = r"split_dicom_data/train/labels"
labels_root = r"Dataset001_MyDataset/labelsTr"

# Make sure output folder exists
os.makedirs(labels_root, exist_ok=True)

# Patient counter
patient_id = 1

# Loop over segmentation files directly in labels folder
for file in sorted(os.listdir(seg_root)):
    if file.endswith(('.nii', '.nii.gz')):
        src_path = os.path.join(seg_root, file)

        # Rename to nnU-Net format
        new_filename = f"la_{patient_id:03d}.nii.gz"
        new_path = os.path.join(labels_root, new_filename)

        # Ensure unique filename if collision
        counter = 1
        while os.path.exists(new_path):
            new_filename = f"la_{patient_id:03d}_{counter}.nii.gz"
            new_path = os.path.join(labels_root, new_filename)
            counter += 1

        # Copy file
        shutil.copy2(src_path, new_path)
        print(f"✅ Copied: {src_path} → {new_path}")

        # Increment ID
        patient_id += 1

print("\n🎯 All label files copied and renamed successfully!")


✅ Copied: split_dicom_data/train/labels/la_002.nii.gz → Dataset001_MyDataset/labelsTr/la_001.nii.gz
✅ Copied: split_dicom_data/train/labels/la_003.nii.gz → Dataset001_MyDataset/labelsTr/la_002.nii.gz
✅ Copied: split_dicom_data/train/labels/la_006.nii.gz → Dataset001_MyDataset/labelsTr/la_003.nii.gz
✅ Copied: split_dicom_data/train/labels/la_007.nii.gz → Dataset001_MyDataset/labelsTr/la_004.nii.gz
✅ Copied: split_dicom_data/train/labels/la_009.nii.gz → Dataset001_MyDataset/labelsTr/la_005.nii.gz
✅ Copied: split_dicom_data/train/labels/la_010.nii.gz → Dataset001_MyDataset/labelsTr/la_006.nii.gz
✅ Copied: split_dicom_data/train/labels/la_011.nii.gz → Dataset001_MyDataset/labelsTr/la_007.nii.gz
✅ Copied: split_dicom_data/train/labels/la_012.nii.gz → Dataset001_MyDataset/labelsTr/la_008.nii.gz
✅ Copied: split_dicom_data/train/labels/la_013.nii.gz → Dataset001_MyDataset/labelsTr/la_009.nii.gz
✅ Copied: split_dicom_data/train/labels/la_014.nii.gz → Dataset001_MyDataset/labelsTr/la_010.nii.gz


Resample segmentation file to match new images

In [ ]:
ref_root = r"Dataset001_MyDataset/imagesTr"
seg_root = r"Dataset001_MyDataset/labelsTr"

for f in os.listdir(ref_root):
    if not f.endswith(".nii.gz"):
        continue

    ref_path = os.path.join(ref_root, f)
    seg_path = os.path.join(seg_root, f.replace("_0000", ""))  # Remove _0000 for label files

    if not os.path.exists(seg_path):
        print(f"No segmentation found for {f}")
        continue

    print(f"Resampling {seg_path} -> match {ref_path}")

    # Load images
    ref = nib.load(ref_path)
    seg = nib.load(seg_path)

    # Resample segmentation to match reference
    seg_resampled = nimg.resample_to_img(seg, ref, interpolation='nearest')

    # Save output
    nib.save(seg_resampled, seg_path)


Resampling Dataset001_MyDataset/labelsTr/la_010.nii.gz -> match Dataset001_MyDataset/imagesTr/la_010_0000.nii.gz


/tmp/ipython-input-1105302431.py:22: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.
  seg_resampled = nimg.resample_to_img(seg, ref, interpolation='nearest')
/tmp/ipython-input-1105302431.py:22: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  seg_resampled = nimg.resample_to_img(seg, ref, interpolation='nearest')


Resampling Dataset001_MyDataset/labelsTr/la_019.nii.gz -> match Dataset001_MyDataset/imagesTr/la_019_0000.nii.gz


/tmp/ipython-input-1105302431.py:22: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.
  seg_resampled = nimg.resample_to_img(seg, ref, interpolation='nearest')
/tmp/ipython-input-1105302431.py:22: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  seg_resampled = nimg.resample_to_img(seg, ref, interpolation='nearest')


Resampling Dataset001_MyDataset/labelsTr/la_018.nii.gz -> match Dataset001_MyDataset/imagesTr/la_018_0000.nii.gz


...

/tmp/ipython-input-1105302431.py:22: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.
  seg_resampled = nimg.resample_to_img(seg, ref, interpolation='nearest')
/tmp/ipython-input-1105302431.py:22: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  seg_resampled = nimg.resample_to_img(seg, ref, interpolation='nearest')


Resampling Dataset001_MyDataset/labelsTr/la_015.nii.gz -> match Dataset001_MyDataset/imagesTr/la_015_0000.nii.gz


/tmp/ipython-input-1105302431.py:22: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.
  seg_resampled = nimg.resample_to_img(seg, ref, interpolation='nearest')
/tmp/ipython-input-1105302431.py:22: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  seg_resampled = nimg.resample_to_img(seg, ref, interpolation='nearest')


Resampling Dataset001_MyDataset/labelsTr/la_014.nii.gz -> match Dataset001_MyDataset/imagesTr/la_014_0000.nii.gz


/tmp/ipython-input-1105302431.py:22: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.
  seg_resampled = nimg.resample_to_img(seg, ref, interpolation='nearest')


Print out 1 sample's channels

Print out 1 sample's intensity range, and a middle slice

In [ ]:
# Load the NIfTI file
img = nib.load("Dataset001_MyDataset/imagesTr/la_021_0000.nii.gz")

# Get the data array
data = img.get_fdata()   # shape: (X, Y, Z)

print("Shape:", data.shape)
print("Intensity range:", np.min(data), "to", np.max(data))

# Plot histogram of intensities
plt.hist(data.flatten(), bins=200)
plt.xlabel("Intensity")
plt.ylabel("Voxel count")
plt.title("Intensity distribution")
plt.show()

# Show a middle slice
mid_slice = data[:, :, data.shape[2]//2]
plt.imshow(mid_slice.T, cmap="gray", origin="lower")
plt.colorbar(label="Intensity")
plt.show()


We will now apply clipping and normalization to the images' channels

In [ ]:
import SimpleITK as sitk
import numpy as np
import os
from glob import glob

def preprocess_image(input_path, output_path):
    """
    Apply only 5th–99.5th percentile clipping.
    """
    # Load
    img = sitk.ReadImage(input_path)
    arr = sitk.GetArrayFromImage(img)  # (z, y, x)

    # --- Step 1: Clip intensities ---
    low, high = np.percentile(arr, [5, 99.5])
    arr = np.clip(arr, low, high)

    # Put back into image
    img_clipped = sitk.GetImageFromArray(arr)
    img_clipped.CopyInformation(img)

    # Save result
    sitk.WriteImage(img_clipped, output_path)

def normalize_image(input_path, output_path, method="minmax"):
    """
    Normalize image intensities.
    method = "minmax" → scale to [0,1]
    method = "zscore" → zero mean, unit variance
    """
    # Load
    img = sitk.ReadImage(input_path)
    arr = sitk.GetArrayFromImage(img)  # (z, y, x)

    # --- Step: Normalize ---
    if method == "minmax":
        arr = (arr - arr.min()) / (arr.max() - arr.min() + 1e-8)
    elif method == "zscore":
        arr = (arr - arr.mean()) / (arr.std() + 1e-8)
    else:
        raise ValueError("method must be 'minmax' or 'zscore'")

    # Save result
    img_norm = sitk.GetImageFromArray(arr.astype(np.float32))
    img_norm.CopyInformation(img)
    sitk.WriteImage(img_norm, output_path)

In [ ]:
# 🔹 Apply to dataset
input_dir = "Dataset001_MyDataset/imagesTr"
output_dir = "Dataset001_MyDataset_Preprocessed/imagesTr"
os.makedirs(output_dir, exist_ok=True)

for path in glob(os.path.join(input_dir, "*.nii.gz")):
    fname = os.path.basename(path)
    out_path = os.path.join(output_dir, fname)
    print(f"Processing {fname} → {out_path}")
    preprocess_image(path, out_path)



In [ ]:
# Load the NIfTI file
img = nib.load("Dataset001_MyDataset_Preprocessed/imagesTr/la_021_0000.nii.gz")

# Get the data array
data = img.get_fdata()   # shape: (X, Y, Z)

print("Shape:", data.shape)
print("Intensity range:", np.min(data), "to", np.max(data))

# Plot histogram of intensities
plt.hist(data.flatten(), bins=200)
plt.xlabel("Intensity")
plt.ylabel("Voxel count")
plt.title("Intensity distribution")
plt.show()

# Show a middle slice
mid_slice = data[:, :, data.shape[2]//2]
plt.imshow(mid_slice.T, cmap="gray", origin="lower")
plt.colorbar(label="Intensity")
plt.show()


In [ ]:
# 🔹 Apply to dataset
input_dir = "Dataset001_MyDataset_Preprocessed/imagesTr"
output_dir = "Dataset001_MyDataset_Preprocessed/imagesTr"
os.makedirs(output_dir, exist_ok=True)

for path in glob(os.path.join(input_dir, "*.nii.gz")):
    fname = os.path.basename(path)
    out_path = os.path.join(output_dir, fname)
    print(f"Processing {fname} → {out_path}")
    normalize_image(path, out_path, method="minmax")  # change to "zscore" if needed


We can see the intensity range is now cleaner

In [ ]:
# Load the NIfTI file
img = nib.load("Dataset001_MyDataset_Preprocessed/imagesTr/la_021_0000.nii.gz")

# Get the data array
data = img.get_fdata()   # shape: (X, Y, Z)

print("Shape:", data.shape)
print("Intensity range:", np.min(data), "to", np.max(data))

# Plot histogram of intensities
plt.hist(data.flatten(), bins=200)
plt.xlabel("Intensity")
plt.ylabel("Voxel count")
plt.title("Intensity distribution")
plt.show()

# Show a middle slice
mid_slice = data[:, :, data.shape[2]//2]
plt.imshow(mid_slice.T, cmap="gray", origin="lower")
plt.colorbar(label="Intensity")
plt.show()


In [ ]:
lbl = nib.load("Dataset001_MyDataset_Preprocessed/imagesTr/la_001_0000.nii.gz").get_fdata()
print(np.unique(lbl))

Move the labelsTr into the Preprocessed folder

In [ ]:
src = r"Dataset001_MyDataset/labelsTr"   # the folder containing subfolders
dst = r"Dataset001_MyDataset_Preprocessed/labelsTr"

# make sure destination exists
os.makedirs(os.path.dirname(dst), exist_ok=True)

# copy labelsTr
shutil.copytree(src, dst, dirs_exist_ok=True)

print(f"Copied {src} → {dst}")

Create the dataset.json file (required for nnUNet training)

In [ ]:
# Paths to your nnUNet dataset folders
imagesTr = r"Dataset001_MyDataset_Preprocessed/imagesTr"
labelsTr = r"Dataset001_MyDataset_Preprocessed/labelsTr"

# Collect all training cases
training_cases = []
for fname in sorted(os.listdir(imagesTr)):
    if fname.endswith("_0000.nii.gz"):  # only pick modality-0 images
        # nnU-Net expects image and label paths relative to dataset root
        image_path = f"./imagesTr/{fname}"
        label_fname = fname.replace("_0000", "")  # strip _0000
        label_path = f"./labelsTr/{label_fname}"

        if os.path.exists(os.path.join(labelsTr, label_fname)):
            training_cases.append({
                "image": image_path,
                "label": label_path
            })
        else:
            print(f"No label found for {fname}")

# Build dataset.json content
dataset = {
    "name": "MySegmentationDataset",
    "description": "Segmentation dataset",
    "tensorImageSize": "3D",
    "channel_names": {
        "0": "CT"  # change to MRI if your scans are MRIs
    },
    "labels": {
        "background": 0,
        "tumor": 1  # add more labels if needed
    },
    "file_ending": ".nii.gz",
    "numTraining": len(training_cases),
    "numTest": 0,
    "training": training_cases,
    "test": []
}

# Save dataset.json
with open("Dataset001_MyDataset_Preprocessed/dataset.json", "w") as f:
    json.dump(dataset, f, indent=4)

print(f"dataset.json created with {len(training_cases)} training cases")


Clone the nnUNet model

In [ ]:
!git clone https://github.com/MIC-DKFZ/nnUNet.git
%cd nnUNet
!pip install -e .


Create folders for nnUNet to use

In [ ]:
import os
os.makedirs("nnUNet_preprocessed", exist_ok=True)
os.makedirs("nnUNet_results", exist_ok=True)
os.makedirs("nnUNet_raw", exist_ok=True)


os.environ['nnUNet_raw'] = f"nnUNet_raw"
os.environ['nnUNet_preprocessed'] = f"nnUNet_preprocessed"
os.environ['nnUNet_results'] = f"nnUNet_results"

Move our preprocessed dataset into nnUNet_raw

In [ ]:
src = r"Dataset001_MyDataset_Preprocessed"   # the folder containing subfolders
dst = r"nnUNet_raw/Dataset001_MyDataset"

# make sure destination exists
os.makedirs(os.path.dirname(dst), exist_ok=True)

# copy labelsTr
shutil.copytree(src, dst, dirs_exist_ok=True)

print(f"Copied {src} → {dst}")

Perform nnUNet's preprocessing

In [ ]:
!nnUNetv2_plan_and_preprocess -d 1 -c 3d_fullres --verify_dataset_integrity

Start training on the whole dataset

In [ ]:
!nnUNetv2_train 001 3d_fullres 0 --c

In [ ]:
!nnUNetv2_train 001 3d_fullres 1 --c

Keep this here just for reference

In [ ]:
!nnUNetv2_train 001 3d_fullres all --c

In [ ]:
import os
import time
import shutil

folder = "1. U nhú - raw"
now = time.time()

# Walk through all files and update timestamps
for root, dirs, files in os.walk(folder):
    for f in files:
        full_path = os.path.join(root, f)
        os.utime(full_path, (now, now))


Save results

In [ ]:
from google.colab import files


# Create a zip file from the preprocessed folder
shutil.make_archive("nnUNet_raws", 'zip', "1. U nhú - raw")
files.download("nnUNet_raws.zip")


In [ ]:
from google.colab import files

files.download("nnUNet_results/Dataset001_MyDataset/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_0/checkpoint_final.pth")


In [ ]:
# Create a zip file from the preprocessed folder
shutil.make_archive("nnUNet_labels", 'zip', "Dataset001_MyDataset/labelsTr")
files.download("nnUNet_labels.zip")


In [ ]:
# Create a zip file from the preprocessed folder
shutil.make_archive("nnUNet_results", 'zip', "nnUNet_results")
files.download("nnUNet_results.zip")


In [ ]:
# Create a zip file from the preprocessed folder
shutil.make_archive("nnUNet_preprocessed", 'zip', "nnUNet_preprocessed")
files.download("nnUNet_preprocessed.zip")


In [ ]:
# Create a zip file from the preprocessed folder
shutil.make_archive("nnUNet_raw", 'zip', "nnUNet_raw")
files.download("nnUNet_raw.zip")


In [ ]:
!nnUNetv2_train 1 3d_fullres 0

Testing:
Download dataset 2 for testing on model trained on dataset 1

In [ ]:
# Google Drive file ID (from the shared link)
file_id = "1GKpXmcqbefmf7x4wtqGwUEX8YiU9l7k9"
file_name = "new_dataset.zip"
file_path = "/content/" + file_name

# Download the file
gdown.download(f"https://drive.google.com/uc?id={file_id}", file_name, quiet=False)
print(f"Downloaded: {file_path}")

In [ ]:
!unzip new_dataset.zip -d new_dataset/

Perform exact preprocessing steps like before training

In [ ]:
# Root containing all patient folders (22..., 23..., etc.)
seg_root = r"new_dataset/Các ca đã label"
labels_root = r"Dataset002_NewDataset/labelsTr"

os.makedirs(labels_root, exist_ok=True)

patient_id = 1

# Loop over patient folders (22..., 23..., 24...)
for patient_folder in sorted(os.listdir(seg_root)):
    patient_path = os.path.join(seg_root, patient_folder)
    if not os.path.isdir(patient_path):
        continue

    # Inside each patient → look for STxxxx
    for study_folder in sorted(os.listdir(patient_path)):
        study_path = os.path.join(patient_path, study_folder)
        if not os.path.isdir(study_path):
            continue

        # Inside study → look for SExxxx
        for series_folder in sorted(os.listdir(study_path)):
            series_path = os.path.join(study_path, series_folder)
            if not os.path.isdir(series_path):
                continue

            print(f"Processing: {series_path}")

            # Collect nii/nii.gz files inside
            nii_files = [
                os.path.join(series_path, f)
                for f in sorted(os.listdir(series_path))
                if f.endswith(('.nii', '.nii.gz'))
            ]

            if nii_files:
                # Take only the first nii file
                src_path = nii_files[0]

                # Rename into nnU-Net format
                new_filename = f"la_{patient_id:03d}.nii.gz"
                new_path = os.path.join(labels_root, new_filename)

                # Avoid collision
                counter = 1
                while os.path.exists(new_path):
                    new_filename = f"la_{patient_id:03d}_{counter}.nii.gz"
                    new_path = os.path.join(labels_root, new_filename)
                    counter += 1

                shutil.move(src_path, new_path)
                print(f"Moved: {src_path} -> {new_path}")

                # Increase patient counter
                patient_id += 1

            # Only one segmentation per patient → stop after first series
            break


In [ ]:
# Input: folder containing all patient DICOM folders
dicom_root = r"new_dataset/Các ca đã label"
output_root = r"Dataset002_NewDataset/imagesTr"

In [ ]:
for patient in sorted(os.listdir(dicom_root)):
    patient_path = os.path.join(dicom_root, patient)
    if os.path.isdir(patient_path):
        print(f"Preparing {patient}...")

        # Flatten DICOMs into patient folder
        flatten_patient_folder(patient_path)

In [ ]:
# Input: folder containing all patient DICOM folders
dicom_root = r"new_dataset/Các ca đã label"
output_root = r"Dataset002_NewDataset/imagesTr"

# Create output folder if not exists
os.makedirs(output_root, exist_ok=True)

# Patient counter
patient_id = 1

for patient in sorted(os.listdir(dicom_root)):
    patient_path = os.path.join(dicom_root, patient)
    if os.path.isdir(patient_path):
        print(f"Converting {patient}...")

        # Temporary output (dcm2niix will dump here)
        temp_out = os.path.join(output_root, "tmp")
        os.makedirs(temp_out, exist_ok=True)

        # Run dcm2niix
        subprocess.run([
            "dcm2niix",
            "-z", "y",              # gzip compression
            "-o", temp_out,         # output folder (temporary)
            patient_path            # input DICOM folder
        ])

        # Find the converted .nii.gz file
        for f in os.listdir(temp_out):
            if f.endswith(".nii") or f.endswith(".nii.gz"):
                old_path = os.path.join(temp_out, f)

                # Rename to nnU-Net format
                new_filename = f"la_{patient_id:03d}_0000.nii.gz"
                new_path = os.path.join(output_root, new_filename)

                # Ensure gzip compressed
                if not old_path.endswith(".nii.gz"):
                    new_path_tmp = new_path.replace(".nii.gz", ".nii")
                    shutil.move(old_path, new_path_tmp)
                    subprocess.run(["gzip", new_path_tmp])
                else:
                    shutil.move(old_path, new_path)

                print(f"Renamed to {new_filename}")

        # Clean up temp
        shutil.rmtree(temp_out)

        patient_id += 1


In [ ]:
ref_root = r"Dataset002_NewDataset/imagesTr"
seg_root = r"Dataset002_NewDataset/labelsTr"

for f in os.listdir(ref_root):
    if not f.endswith(".nii.gz"):
        continue

    ref_path = os.path.join(ref_root, f)
    seg_path = os.path.join(seg_root, f.replace("_0000", ""))  # Remove _0000 for label files

    if not os.path.exists(seg_path):
        print(f"No segmentation found for {f}")
        continue

    print(f"Resampling {seg_path} -> match {ref_path}")

    # Load images
    ref = nib.load(ref_path)
    seg = nib.load(seg_path)

    # Resample segmentation to match reference
    seg_resampled = nimg.resample_to_img(seg, ref, interpolation='nearest')

    # Save output
    nib.save(seg_resampled, seg_path)


In [ ]:
# 🔹 Apply to dataset
input_dir = "Dataset002_NewDataset/imagesTr"
output_dir = "Dataset002_NewDataset_Preprocessed/imagesTr"
os.makedirs(output_dir, exist_ok=True)

for path in glob(os.path.join(input_dir, "*.nii.gz")):
    fname = os.path.basename(path)
    out_path = os.path.join(output_dir, fname)
    print(f"Processing {fname} → {out_path}")
    preprocess_image(path, out_path)
    normalize_image(path, out_path, method="minmax")  # change to "zscore" if needed



In [ ]:
# Load the NIfTI file
img = nib.load("Dataset002_NewDataset_Preprocessed/imagesTr/la_028_0000.nii.gz")

# Get the data array
data = img.get_fdata()   # shape: (X, Y, Z)

print("Shape:", data.shape)
print("Intensity range:", np.min(data), "to", np.max(data))

# Plot histogram of intensities
plt.hist(data.flatten(), bins=200)
plt.xlabel("Intensity")
plt.ylabel("Voxel count")
plt.title("Intensity distribution")
plt.show()

# Show a middle slice
mid_slice = data[:, :, data.shape[2]//2]
plt.imshow(mid_slice.T, cmap="gray", origin="lower")
plt.colorbar(label="Intensity")
plt.show()


Download nnU_results to infer using this model

In [ ]:
import SimpleITK as sitk
import os
from glob import glob

def histogram_match_image(moving_image, reference_image, bins=256, match_points=50):
    """
    Adjust intensity distribution of moving_image to match reference_image.
    """
    matcher = sitk.HistogramMatchingImageFilter()
    matcher.SetNumberOfHistogramLevels(bins)
    matcher.SetNumberOfMatchPoints(match_points)
    matcher.ThresholdAtMeanIntensityOn()
    return matcher.Execute(moving_image, reference_image)

# Paths
input_dir = r"Dataset002_NewDataset_Preprocessed/imagesTr"
output_dir = r"Dataset001_NewDataset_Preprocessed/imagesTr"
os.makedirs(output_dir, exist_ok=True)

# Pick a reference image (the one that looks "good")
reference_path = os.path.join(input_dir, "la_001_0000.nii.gz")  # <-- change if needed
reference_img = sitk.ReadImage(reference_path)

# Loop over all .nii.gz images
for path in glob(os.path.join(input_dir, "*.nii.gz")):
    fname = os.path.basename(path)
    out_path = os.path.join(output_dir, fname)

    print(f"Processing {fname} → {out_path}")

    moving_img = sitk.ReadImage(path)
    matched_img = histogram_match_image(moving_img, reference_img)

    sitk.WriteImage(matched_img, out_path)

print("✅ Histogram matching completed for all images.")


In [ ]:
import gdown
# Google Drive file ID (from the shared link)
file_id = "17NhcFTSrZWpK94nt23paT49N_ly_OZjH"
file_name = "nnU_results.zip"
file_path = "/content/" + file_name

# Download the file
gdown.download(f"https://drive.google.com/uc?id={file_id}", file_name, quiet=False)
print(f"Downloaded: {file_path}")

In [ ]:
!unzip nnU_results.zip -d nnUNet_results/

In [ ]:
src = r"Dataset002_NewDataset/labelsTr"   # the folder containing subfolders
dst = r"Dataset002_NewDataset_Preprocessed/labelsTr"

# make sure destination exists
os.makedirs(os.path.dirname(dst), exist_ok=True)

# copy labelsTr
shutil.copytree(src, dst, dirs_exist_ok=True)

print(f"Copied {src} → {dst}")

In [ ]:
import json

# Paths to your nnUNet dataset folders
imagesTr = r"Dataset002_NewDataset_Preprocessed/imagesTr"
labelsTr = r"Dataset002_NewDataset_Preprocessed/labelsTr"


# Collect all training cases
training_cases = []
for fname in sorted(os.listdir(imagesTr)):
    if fname.endswith(".nii.gz"):
        # nnU-Net expects that image file names end with _0000.nii.gz
        image_path = f"./imagesTr/{fname}"
        label_path = f"./labelsTr/{fname.replace('_0000', '')}"

        if os.path.exists(os.path.join(labelsTr, f"{fname.replace('_0000', '')}")):
            training_cases.append({
                "image": image_path,
                "label": label_path
            })
        else:
            print(f"Warning: No label found for {fname}")

# Build dataset.json content
dataset = {
    "name": "MySegmentationDataset",
    "description": "Segmentation dataset",
    "tensorImageSize": "3D",
    "channel_names": {
        "0": "CT"  # change to MRI if your scans are MRIs
    },
    "labels": {
        "background": 0,
        "tumor": 1  # add more labels if needed
    },
    "file_ending": ".nii.gz",
    "numTraining": len(training_cases),
    "numTest": 0,
    "training": training_cases,
    "test": []
}

# Save dataset.json
with open("Dataset002_NewDataset_Preprocessed/dataset.json", "w") as f:
    json.dump(dataset, f, indent=4)

print(f"dataset.json created with {len(training_cases)} training cases")


Prediction

In [ ]:
!nnUNetv2_predict \
    -i Dataset002_NewDataset_Preprocessed/imagesTr \
    -o test_predictions \
    -d 1 \
    -c 3d_fullres \
    -f all

Check dice of results

In [ ]:
import os
import SimpleITK as sitk
import numpy as np

def dice_coefficient(gt_file, pred_file, label=1):
    gt = sitk.GetArrayFromImage(sitk.ReadImage(gt_file))
    pred = sitk.GetArrayFromImage(sitk.ReadImage(pred_file))

    gt_bin = (gt == label).astype(np.uint8)
    pred_bin = (pred == label).astype(np.uint8)

    intersection = np.sum(gt_bin * pred_bin)
    denominator = np.sum(gt_bin) + np.sum(pred_bin)

    if denominator == 0:
        return 1.0 if np.sum(gt_bin) == 0 and np.sum(pred_bin) == 0 else 0.0

    return 2.0 * intersection / denominator


# Paths
gt_root = "Dataset002_NewDataset_Preprocessed/labelsTr"
pred_root = "test_predictions"
dices = []

# Loop through predictions
for pred_file in sorted(os.listdir(pred_root)):
    if not pred_file.endswith(".nii.gz"):
        continue

    # Predictions usually named like la_001.nii.gz
    gt_file = os.path.join(gt_root, pred_file)
    pred_path = os.path.join(pred_root, pred_file)

    if not os.path.exists(gt_file):
        print(f"No ground truth for {pred_file}, skipping.")
        continue

    dice = dice_coefficient(gt_file, pred_path, label=1)
    dices.append(dice)
    print(f"{pred_file}: Dice = {dice:.4f}")
print(f"Mean Dice: {np.mean(dices):.4f}")


In [ ]:
import nibabel as nib
import os
import nilearn.image as nimg

ref_root = r"Dataset002_NewDataset/imagesTr"
pred_root = r"test_predictions"
out_root = r"aligned_predictions"

os.makedirs(out_root, exist_ok=True)

for f in os.listdir(ref_root):
    if not f.endswith(".nii.gz") or "_0000" not in f:
        continue

    ref_path = os.path.join(ref_root, f)
    pred_path = os.path.join(pred_root, f.replace("_0000", ""))
    out_path = os.path.join(out_root, f.replace("_0000", ""))

    if not os.path.exists(pred_path):
        print(f"No prediction for {f}")
        continue

    print(f"Aligning {pred_path} -> {ref_path}")

    # Load reference (original DICOM-converted) and prediction
    ref_img = nib.load(ref_path)
    pred_img = nib.load(pred_path)

    # Resample prediction into reference voxel grid
    pred_resampled = nimg.resample_to_img(
        pred_img, ref_img, interpolation="nearest"
    )

    # Force reference affine & header (keeps exact orientation/position)
    pred_resampled = nib.Nifti1Image(
        pred_resampled.get_fdata().astype("int16"),
        ref_img.affine,
        ref_img.header
    )

    nib.save(pred_resampled, out_path)
    print(f"Saved: {out_path}")


In [ ]:
!pip install pydicom nibabel


This is for download results, we resample the predicted segmentation again (to suit the dicom files)

In [ ]:
import os
import nibabel as nib
import numpy as np
from nilearn.image import resample_img

# Folders
ref_root = "reflabelsTr"   # folder containing reference NIfTIs (from dcm2niix)
pred_root = "test_predictions"      # nnU-Net predictions
out_root = "test_predictions_aligned"
os.makedirs(out_root, exist_ok=True)

for pred_file in sorted(os.listdir(pred_root)):
    if not pred_file.endswith(".nii.gz"):
        continue

    pred_path = os.path.join(pred_root, pred_file)

    # Find corresponding reference file (assuming similar naming e.g. la_001 -> la_001_ref.nii.gz)
    # Adjust this rule if your reference naming differs!
    ref_name = pred_file.replace(".nii.gz", ".nii.gz")
    ref_path = os.path.join(ref_root, ref_name)

    if not os.path.exists(ref_path):
        print(f"⚠️ No reference found for {pred_file}, skipping.")
        continue

    # Load reference and prediction
    ref = nib.load(ref_path)
    pred = nib.load(pred_path)

    # Resample prediction into reference grid
    pred_resampled = resample_img(
        pred,
        target_affine=ref.affine,
        target_shape=ref.shape,
        interpolation="nearest"   # keep labels discrete
    )

    # Save aligned segmentation
    out_path = os.path.join(out_root, pred_file.replace(".nii.gz", "_aligned_to_ref.nii.gz"))
    aligned_seg = nib.Nifti1Image(pred_resampled.get_fdata().astype(np.uint8), ref.affine, ref.header)
    nib.save(aligned_seg, out_path)

    print(f"✅ Saved aligned segmentation: {out_path}")


In [ ]:
import shutil

# Zip the aligned folder
shutil.make_archive("test_predictions_aligned", 'zip', "test_predictions_aligned")

print("✅ Zipped folder: test_predictions_aligned.zip")

In [ ]:
from google.colab import files
files.download("test_predictions_aligned.zip")

In [ ]:
!sudo apt-get install -y dcmqi


In [ ]:
!itkimage2segimage \
    --inputImageList test_predictions/la_001.nii.gz \
    --inputDICOMDirectory 'new_dataset/ĐỢT 2 (tháng 9 2025)/Các ca đã label/22 - BUI DUC NAM/ST000000/SE000000' \
    --outputDICOM la_001.nii.gz \
    --segmentAlgorithmName nnUNet \
    --segmentLabel "Left Atrium"
